### 5. Metin Verisi Üzerinde Uygulama

Bu bölümde, **gerçek bir Türkçe metin veri setiyle tam bir NLP uygulaması** adım adım gerçekleştirilecek. Amaç, yazılım geliştiricilerin teoriden uygulamaya geçiş yapmasını sağlamak ve veri ön işleme, model eğitimi, değerlendirme ve entegrasyon süreçlerini yaşamasını sağlamaktır.

### 1. Veri Hazırlığı

İlk adım, kaliteli bir veri seti oluşturmaktır. HuggingFace `datasets` kütüphanesi ile örnek bir veri seti yükleyebiliriz.

In [ ]:
from datasets import load_dataset
import torch
dataset = load_dataset("winvoker/turkish-sentiment-analysis-dataset")

### 2. Veri Ön İşleme

Metinlerdeki gereksiz karakterler, emoji, URL'ler temizlenmeli. Ayrıca küçük harfe çevirmek ve özel karakterleri çıkarmak önemlidir.

In [ ]:
def filter_neutral(examples):
    return examples["label"] != "Notr"
dataset = dataset.filter(filter_neutral)
def reformat_labels(examples):
    examples["label"] = 1 if examples["label"] == "Positive" else 0
    return examples

dataset = dataset.map(reformat_labels)

In [ ]:
print(dataset["train"][0])

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

### 3. Tokenization

Yine HuggingFace tokenizer’ı kullanıyoruz.

In [ ]:
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=128)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

### 4. Model Eğitimi (Fine-Tuning)

Daha önce anlatılan `Trainer` yapısını kullanıyoruz.

In [ ]:
from transformers import TrainingArguments, Trainer
tranging_args = TrainingArguments(
    output_dir="./bertfinetuningornek",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=tranging_args,
    train_dataset=tokenized_datasets["train"]
)

In [ ]:
trainer.train()

5. Modeli Kaydet ve Yükle

In [ ]:
import numpy as np
text = "Bu restoran çok kötü idi"
inputs = tokenizer(text,return_tensors="pt",truncation=True,padding=True,max_length=128)
outputs = model(**inputs)
prediction = torch.nn.functional.softmax(outputs.logits,dim=-1)
predicted_class = np.argmax(prediction.detach().numpy())
predicted_class

6. Gerçek Zamanlı Tahmin